In [85]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import os
import random

In [86]:
random.seed(42)

In [87]:
os.getcwd()

'/Users/luisaparys/Desktop/BDCC'

In [88]:
df = pd.read_parquet('est_hourly.paruqet')

In [89]:
data = pd.DataFrame()

In [90]:
data["1h_lag"] = df["PJME"].shift(1)
data["24h_lag"] = df["PJME"].shift(24)
data["target"] = (df["PJME"] > np.quantile(df["PJME"].dropna(), 0.9))

In [91]:
data.dropna(inplace=True)

In [92]:
data

,1h_lag,24h_lag,target
Datetime,,,
2002-12-30 01:00:00,27008.0,26498.0,False
2002-12-30 02:00:00,27526.0,25147.0,False
2002-12-30 03:00:00,26600.0,24574.0,False
2002-12-30 04:00:00,26241.0,24393.0,False
2002-12-30 05:00:00,26213.0,24860.0,False
...,...,...,...
2018-01-01 20:00:00,44343.0,45787.0,True
2018-01-01 21:00:00,44284.0,45209.0,True
2018-01-01 22:00:00,43751.0,43663.0,True


In [93]:
data1 = data[0:50000]
data2 = data[50000:100000]
data3 = data[100000:]

model0 = LogisticRegression()
model1 = LogisticRegression()
model2 = LogisticRegression()
model3 = LogisticRegression()

model0.fit(data.drop(columns=["target"]), data["target"])
model1.fit(data1.drop(columns=["target"]), data1["target"])
model2.fit(data2.drop(columns=["target"]), data2["target"])
model3.fit(data3.drop(columns=["target"]), data3["target"])

LogisticRegression()

In [94]:
def make_Ak_matrix(coefs, inter, x):
    
    expon = np.exp(inter[0] + coefs[0][0]*x[0] + coefs[0][1]*x[1])
    mat = np.array([
        [     expon/((expon + 1)**2),      x[0]*expon/((expon + 1)**2),      x[1]*expon/((expon + 1)**2)],
        [x[0]*expon/((expon + 1)**2), x[0]*x[0]*expon/((expon + 1)**2), x[0]*x[1]*expon/((expon + 1)**2)],
        [x[1]*expon/((expon + 1)**2), x[0]*x[1]*expon/((expon + 1)**2), x[1]*x[1]*expon/((expon + 1)**2)]
    ])
    return mat

In [95]:
mat1 = np.zeros((3,3))
mat2 = np.zeros((3,3))
mat3 = np.zeros((3,3))

for i in range(len(data1)):
    temp_mat = make_Ak_matrix(model1.coef_, model1.intercept_, list(data1.drop(columns=["target"]).iloc[i]))
    mat1 -= temp_mat
    
for i in range(len(data2)):
    temp_mat = make_Ak_matrix(model2.coef_, model2.intercept_, list(data2.drop(columns=["target"]).iloc[i]))
    mat2 -= temp_mat
    
for i in range(len(data3)):
    temp_mat = make_Ak_matrix(model3.coef_, model3.intercept_, list(data3.drop(columns=["target"]).iloc[i]))
    mat3 -= temp_mat
    

In [96]:
side_mat = np.matmul(mat1, np.insert(model1.coef_, 0, model1.intercept_).reshape(3,1)) + np.matmul(mat2, np.insert(model2.coef_, 0, model2.intercept_).reshape(3,1)) + np.matmul(mat3, np.insert(model3.coef_, 0, model3.intercept_).reshape(3,1))

In [97]:
np.matmul(np.linalg.inv(mat1 + mat2 + mat3), side_mat)

array([[-4.99739037e+01],
       [ 1.01874523e-03],
       [ 2.10697082e-04]])

In [98]:
model0.coef_

array([[0.00102097, 0.00021094]])

In [99]:
model0.intercept_

array([-50.07296761])